# 美麗的資料探勘

本篇將會簡單的講述我們如何把資料統整、製圖、製成網頁

可以觀賞我們的成果網頁：https://weitheshinobi.github.io/data-mining-project/

In [1]:
from math import *
import pandas as pd
import numpy as np

## 取得景點的天氣

我們從「政府資料開放平台」和「觀測資料查詢 CODiS」下載了遊客、景點、氣象觀測站的資料。

我們之所以下載觀測站的資料，是因為想要知道景點的天氣，但我們不知道每個景點最接近的觀測站是哪個，所以我們可以自己動手計算。

In [2]:
pd.read_csv('static/景點.csv')[0:5]

,編號,地點,地址,緯度,經度,類別
0,1,臺灣鹽博物館,724 臺南市七股區鹽埕里69號,23.15535,120.10565,地方展館、在地藝文、無障礙設施
1,2,七股鹽山,724 臺南市七股區鹽埕里66號,23.15427,120.10026,生態教育、自然景觀、休閒農漁、無障礙設施
2,3,北門遊客中心,727 臺南市北門區北門里舊埕200號,23.26715,120.12397,在地藝文、無障礙設施
3,4,井仔腳瓦盤鹽田,727 臺南市北門區永華里井仔腳興安宮旁,23.25958,120.10819,生態教育、自然景觀、無障礙設施
4,5,尖山埤江南渡假村,736 臺南市柳營區旭山里60號,23.26649,120.39331,生態教育


In [3]:
pd.read_csv('static/觀測站.csv')[0:3]

,站號,站名,海拔高度(m),經度,緯度,城市,地址,資料起始日期,撤站日期,備註,Unnamed: 10
0,C0O810,曾文,161.0,120.497328,23.219681,臺南市,楠西區曾文新村(曾文水庫管理局內),1998/10/8,NaN,NaN,NaN
1,C0O830,北寮,127.0,120.495008,23.079561,臺南市,南化區北寮里48-4號,2013/4/18,NaN,原(C1O830)站因擴充為氣象站，於2013/4/18升級為(C0O830)站。,C1O830
2,C0O840,王爺宮,134.0,120.400850,23.222064,臺南市,六甲區王爺里24號（六甲國小湖東分校）,2013/12/1,NaN,原(C1O840)站因觀測條件不佳及擴充為氣象站，於2013/12/1升級為(C0O840)站。,C1O840


景點和觀測站都有經緯度的資料，這樣很好，因為我們可以從這個來計算距離，距離最短的站就是。

In [4]:
def __compute_distance(lat1, lng1, lat2, lng2):
    radlat1 = radians(lat1)
    radlat2 = radians(lat2)
    a = radlat1 - radlat2
    b = radians(lng1) - radians(lng2)
    s = 2 * asin(sqrt(pow(sin(a / 2), 2) + cos(radlat1) * cos(radlat2) * pow(sin(b / 2), 2)))
    earth_radius = 6378.137
    s = s * earth_radius
    if s < 0:
        return -s
    else:
        return s
    
# 測試 單位為公里
__compute_distance(120, 23, 121, 24)

124.8331458033635

這是透過經緯度計算距離的函式。

In [5]:
def __get_distance_point_to_site(point_lat, point_lng):
    sites = pd.read_csv('static/觀測站.csv')
    sites_name = sites['站名'].tolist()
    sites_lat = sites['經度'].tolist()
    sites_lng = sites['緯度'].tolist()

    tmp_for_search_name = {}
    tmp_distance = []
    for i, site_name in enumerate(sites_name):
        distance = __compute_distance(point_lat, point_lng, sites_lat[i], sites_lng[i])
        tmp_for_search_name[distance] = site_name
        tmp_distance.append(distance)
    nearest_site_distance = np.min(tmp_distance)
    nearest_site_name = tmp_for_search_name[nearest_site_distance]

    return [nearest_site_name, round(nearest_site_distance, 3)]
# 測試
__get_distance_point_to_site(120, 23)

['七股', 12.632]

輸入景點的經緯度，
輸出最近的觀測站與距離（公里）。

In [6]:
def create_nearest_site_csv():
    points = pd.read_csv('static/景點.csv')
    points_name = points['地點'].tolist()
    points_lat = points['經度'].tolist()
    points_lng = points['緯度'].tolist()

    csv_data = []
    for i, point_name in enumerate(points_name):
        tmp = [point_name]
        tmp.extend(__get_distance_point_to_site(points_lat[i], points_lng[i]))
        csv_data.append(tmp)
    header = ["景點", "最近觀測站", "距離(km)"]
    pd.DataFrame(csv_data).to_csv('output/site_distance.csv', header=header, index=0, encoding='big5')

    print("檔案 site_distance.csv 已創建")
    
pd.read_csv('output/site_distance.csv',encoding='big5')[0:5]

,景點,最近觀測站,距離(km)
0,臺灣鹽博物館,七股,2.206
1,七股鹽山,七股,1.607
2,北門遊客中心,北門,0.172
3,井仔腳瓦盤鹽田,北門,1.978
4,尖山埤江南渡假村,東河,1.900


這個函式用到上面幾個函式，
將景點經緯度輸入，
輸出了所有景點最近的觀測站，
並且輸出成表，
我們就能取得每個景點的溫度資料。

整理好的資料就可以進行作圖、分析。

## 整理資料給網頁

準備將製作好的圖表、資料放進網頁中，
我們這次是弄一個簡單的靜態網頁，
所以直接把資料和圖表放上去，

但是還是有一些繁瑣的流程可以讓機器來做。

我們需要在網頁上標記25個點，
這還蠻累人的，
每個點幾乎是一樣的，
除了個別的資料，
這代表我們可以讓機器替我們完成重複性的工作。

我們先讀取會用到的資料。

In [7]:
data = pd.read_csv('static/景點.csv')
point_intro = pd.read_csv('static/景點統整.csv')
position = data['地點']

In [8]:
def get_HTML():
    # 為了讓版面美觀，這裡只顯示三個示範
    for i in range(3):
        print("<button onclick=\"getTouristData(this.value)\" title=" + position[i] + " id=" + position[i] + " value=" +
              position[i] + " class=tourist><i class=\"map marker alternate icon\"></i></button>")


get_HTML()

<button onclick="getTouristData(this.value)" title=臺灣鹽博物館 id=臺灣鹽博物館 value=臺灣鹽博物館 class=tourist><i class="map marker alternate icon"></i></button>
<button onclick="getTouristData(this.value)" title=七股鹽山 id=七股鹽山 value=七股鹽山 class=tourist><i class="map marker alternate icon"></i></button>
<button onclick="getTouristData(this.value)" title=北門遊客中心 id=北門遊客中心 value=北門遊客中心 class=tourist><i class="map marker alternate icon"></i></button>




我們只寫了一次就讓Python幫我們弄好了，
HTML替我們生成了點，
現在CSS替我們標記位置。

生成CSS也是同理。

In [9]:
def get_point_css():
    # 為了讓版面美觀，這裡只顯示三個示範
    for i in range(3):
        print("#" + position[i] + " {")
        print("    left: " + str(i) + "%;")
        print("    top: " + str(i) + "%;")
        print("}")
        print()

get_point_css()

#臺灣鹽博物館 {
    left: 0%;
    top: 0%;
}

#七股鹽山 {
    left: 1%;
    top: 1%;
}

#北門遊客中心 {
    left: 2%;
    top: 2%;
}



得到了點的CSS模板，
當然這些點的參數還是要靠自己動手。

把網頁弄好之後，
可以開始把資料放入，
我們用到一點JavaScript來操控網頁元素，
我會說明一些關於資料的部分。

下方的JavaScript程式碼就是用來操作地圖的，
只要把資料放入即可，
那資料怎麼生成？
當然是靠Python從檔案讀取囉，
千萬不要想手動輸入。

```javascript

// js程式碼
function getTouristData(tourist) {
    $("#tips").text("");
    $("#t-title").text(tourist);
    $("#t-content").html(touristIntro[tourist]);
    $("#t-lat").text(touristLatLng[tourist][0]);
    $("#t-lng").text(touristLatLng[tourist][1]);
    $("#t-type").html(touristType[tourist]);
    $("#temperature-img").attr('src', "static/images/tourist/temperature/" + tourist + ".png")
    $("#tourist-img").attr('src', "static/images/tourist/tourist/" + tourist + ".png")
    $("#rain-img").attr('src', "static/images/tourist/rain/" + tourist + ".png")
}

```

In [10]:
def get_point_intro():
    intro_dict = {}
    for i, intro in enumerate(point_intro['介紹']):
        # 為了讓版面美觀，這裡只顯示一個示範
        if i == 1:
            break
            
        intro_dict[point_intro['名稱'][i]] = "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + intro            
    print(intro_dict)
    
get_point_intro()

{'台灣鹽博物館': '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;一望無際的鹽田中，有兩座白色金字塔並肩聳立，與七股鹽山遙遙相望，這裡是台灣鹽博物館。用視覺回憶鹽田風光；用觸覺掌握鹽粒結晶；用味覺品嘗粗鹽鹹味；用感覺和認知完整體會一路走來的鹽業歷史。'}


看起來是不是很像JSON，
JSON = JavaScript Object Notation
JavaScript物件表示法，

這是巧合嗎？
我不這麼認為，
總之Python的字典直接print就是JSON物件，
這樣我們就能把超長一串寫進我們的網頁JavaScript中。

我們又再次讓機器替我們完成工作。

```JavaScript
// js程式碼
let touristIntro = {"...略"}
```

In [11]:

def get_lat_and_log():
    latlog_dict = {}
    for i, point in enumerate(point_intro['名稱']):
        # 為了讓版面美觀，這裡只顯示三個示範
        if i == 3:
            break
            
        latlog = [point_intro['緯度'][i], point_intro['經度'][i]]
        latlog_dict[point] = latlog
    print(latlog_dict)


get_lat_and_log()

{'台灣鹽博物館': [23.15535, 120.10565], '七股鹽山': [23.15427, 120.10026], '北門遊客中心': [23.26715, 120.12397]}


取得經緯度同理，
這裡簡單示範。

In [12]:
def get_type():
    for i, point in enumerate(point_intro['名稱']):
        # 為了讓版面美觀，這裡只顯示一個示範
        if i == 1:
            break
        
        print("touristType[\"" + point + "\"] = ", end="")
        print("\"", end="")
        for j in point_intro['類別'][i].split('、'):
            print("<div class=\\\"ui basic teal label\\\">", end="")
            print(j, end="")
            print("</div>", end="")
        print("\"", end="")
        print()

get_type()

touristType["台灣鹽博物館"] = "<div class=\"ui basic teal label\">地方展館</div><div class=\"ui basic teal label\">在地藝文</div><div class=\"ui basic teal label\">無障礙設施</div>"


景點標籤細節比較不一樣，
但是精神上是一樣的，
讓機器替我們處理。

最終我們完成網頁地圖點景的資料。

https://weitheshinobi.github.io/data-mining-project/